In [1]:
!nvidia-smi

Tue Jun 28 12:32:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%pip install -q transformers[sentencepiece] datasets hydra-core wandb

In [4]:
import sys
import os

COLAB_ROOT_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/"
sys.path.append(COLAB_ROOT_PATH + "src/util_code")
os.chdir(COLAB_ROOT_PATH)

In [29]:
# Load the config for training run
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import DictConfig, OmegaConf

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="configs")
config = compose(config_name="config")

In [30]:
import colab_utils
import utils
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
from torch.utils.data import DataLoader
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch.multiprocessing as mp
from transformers import logging
import warnings
import wandb

logging.set_verbosity_error()
logging.set_verbosity_warning()
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
# This is where we configure the run parameters
# Model to run
EXP_NUM = "00"
MODEL_NAME = "deberta-v3-small"
TRAIN_RUN_NAME = "baseline"
TRANSFORMER_CHECKPOINT = "microsoft/deberta-v3-small"
EXPERIMENT_NAME = EXP_NUM + "_" + MODEL_NAME + "_" + TRAIN_RUN_NAME

# update the config with run params
# update the output_dir to model specific directory
config.paths.out_dir += MODEL_NAME
print(f"config.paths.out_dir = {config.paths.out_dir}")
config.train_run.transformer_checkpoint = TRANSFORMER_CHECKPOINT
print(f"config.train_run.transformer_checkpoint = {config.train_run.transformer_checkpoint}")
config.train_run.num_workers = mp.cpu_count()
print(f"config.train_run.num_workers = {config.train_run.num_workers}")
config.train_run.experiment_name = EXPERIMENT_NAME
print(f"config.train_run.experiment_name = {config.train_run.experiment_name}")
if config.wandb.enabled:
    # enable reporting to wandb via huggingface training arguments
    config.training_args.report_to = "wandb"
    config.wandb.key = "c5e2877bf080e6b62fcc57231c91e3a1455f97d0"    

config.paths.out_dir = /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v3-small
config.train_run.transformer_checkpoint = microsoft/deberta-v3-small
config.train_run.num_workers = 2
config.train_run.experiment_name = 00_deberta-v3-small_baseline


In [32]:
print(OmegaConf.to_yaml(config))

train_run:
  random_state: 42
  num_labels: 1
  label_col: score
  num_folds: 5
  run_all_folds: false
  num_epochs: 2
  num_workers: 2
  device: cuda
  subset_rows_frac: 0.05
  train_on_subset: false
  transformer_checkpoint: microsoft/deberta-v3-small
  experiment_name: 00_deberta-v3-small_baseline
  save_artifacts: true
wandb:
  key: c5e2877bf080e6b62fcc57231c91e3a1455f97d0
  project: USPPPM
  enabled: true
training_args:
  output_dir: None
  evaluation_strategy: epoch
  save_strategy: epoch
  save_total_limit: 1
  load_best_model_at_end: true
  metric_for_best_model: pearson
  greater_is_better: true
  log_level: warning
  group_by_length: true
  report_to: wandb
  num_train_epochs: 2
  per_device_train_batch_size: 40
  per_device_eval_batch_size: 80
  weight_decay: 0.01
  learning_rate: 4.0e-05
  warmup_ratio: 0.1
  gradient_accumulation_steps: 3
  fp16: true
  lr_scheduler_type: linear
  adam_epsilon: 1.0e-06
paths:
  data_path: /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatc

In [33]:
colab_utils.empty_gdrive_trash()

The below files were cleared from trash
[]


In [34]:
def initialize_wandb(fold):    
    wandb.login(key=config.wandb.key)
    wandb.init(
        config=config,
        project=config.wandb.project,
        group=config.train_run.experiment_name,
        name=f"fold_{fold}"
    )        

In [35]:
df_train = pd.read_csv(config.paths.data_path + "train.csv")
df_train["section"] = df_train.context.str[0]
df_test = pd.read_csv(config.paths.data_path + "test.csv")
df_titles = pd.read_csv(config.paths.data_path + "titles.csv")

In [36]:
from sklearn.preprocessing import LabelEncoder

anchor_encoder = LabelEncoder()
df_train["anchor_map"] = anchor_encoder.fit_transform(df_train["anchor"])
df_train["context_map"] = anchor_encoder.fit_transform(df_train["context"])
df_train["anchor_context_map"] = df_train["anchor_map"].astype(str).str.cat(df_train["context_map"].astype(str), sep="_")
# Score is not really a continuous value here as there are just five distinct values. But since it is float it needs to be converted
# to categorical value before we can perform stratified split on score
df_train["score_map"] = df_train["score"].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})

In [37]:
if config.train_run.train_on_subset:
    print(f"Selecting {config.train_run.subset_rows_frac * 100}% training data")
    df_train = df_train.sample(
        frac=config.train_run.subset_rows_frac, 
        random_state=config.train_run.random_state
    ).reset_index(drop=True)

# Since the target column (score) is continuous, we need to create bins out of the target column
# df_train.loc[:, "bins"] = pd.cut(df_train.score, bins=5, labels=[0,1,2,3,4])
# df_train = utils.strat_kfold_dataframe(df_train, target_col_name="bins", random_state=config.train_run.random_state, num_folds=Config.NUM_FOLDS)

# Now do a stratified group k fold on the bins column (which is a categorical column) and anchor as groups
df_train = utils.strat_group_kfold_dataframe(
    df_train, 
    target_col_name="score_map", 
    group_col_name="anchor_context_map",
    random_state=config.train_run.random_state, 
    num_folds=config.train_run.num_folds
)            
# drop the bin column
# df_train = df_train.drop(["bins"], axis=1)
# df_train = df_train.drop(["anchor_map", "score_map"], axis=1)

In [38]:
# Let us check if the stratification has been done correctly
# The mean of score column should be similar across folds 
fold_score_mean = []
fold_anchor_context_maps = []
for fold in range(config.train_run.num_folds):
    df_train_fold = df_train[df_train.kfold == fold]
    fold_score_mean.append(np.mean(df_train_fold.score.values))
    fold_anchor_context_maps.append(set(df_train_fold.anchor_context_map.unique()))
fold_score_mean

[0.3557356434260165,
 0.35813229056203605,
 0.36268028846153844,
 0.36315899290582837,
 0.37085976039464413]

In [39]:
# check each of the folds has no common anchor value
def check_disjoint(start, fold_anchor_context_maps):
    for i in range(start, 4):
        for j in range(i+1, 5):
            if fold_anchor_context_maps[i].isdisjoint(fold_anchor_context_maps[j]):
                print(f"anchor context map for fold {i} and {j} are disjoint")

check_disjoint(0, fold_anchor_context_maps)                

anchor context map for fold 0 and 1 are disjoint
anchor context map for fold 0 and 2 are disjoint
anchor context map for fold 0 and 3 are disjoint
anchor context map for fold 0 and 4 are disjoint
anchor context map for fold 1 and 2 are disjoint
anchor context map for fold 1 and 3 are disjoint
anchor context map for fold 1 and 4 are disjoint
anchor context map for fold 2 and 3 are disjoint
anchor context map for fold 2 and 4 are disjoint
anchor context map for fold 3 and 4 are disjoint


In [40]:
df_titles.head()

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


In [41]:
df_train = pd.merge(
    left = df_train,
    right = df_titles[["code", "title"]],
    how = "inner",
    left_on = "context",
    right_on = "code"
)

In [42]:
df_train.anchor_context_map.value_counts()

555_89     84
129_47     79
324_49     79
430_104    74
129_48     73
           ..
525_62      1
649_62      1
480_11      1
485_11      1
727_101     1
Name: anchor_context_map, Length: 1699, dtype: int64

In [43]:
# For each anchor, context group (i.e. set of records having same anchor and context values), concatenate the target phrases
# key is unique anchor_context_map , value is concatenation of target phrases of all records for that unique anchor_context_map
anc_ctx_targets = {}
for anchor_context_map in df_train.anchor_context_map.unique():
    df_train_sub = df_train[df_train.anchor_context_map == anchor_context_map]
    anchor_context_target_text = ",".join(df_train_sub.target)    
    anc_ctx_targets[anchor_context_map] = anchor_context_target_text

df_train["anchor_context_targets"] = df_train.anchor_context_map.map(anc_ctx_targets)
df_train["anc_ctx_tgt_len"] = df_train["anchor_context_targets"].apply(lambda text: len(text.split()))
df_train = df_train.sort_values(by=["anc_ctx_tgt_len"], ascending=False)
# df_train = df_train.head(500)
# df_train = df_train[df_train.anchor_context_map == "555_89"]

In [44]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(config.train_run.transformer_checkpoint)
# DataCollatorWithPadding pads each batch to the longest sequence length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
df_train['sectok'] = '[' + df_train.section + ']'
sectoks = list(df_train.sectok.unique())
print(f"Additional special tokens: {sectoks}")
tokenizer.add_special_tokens({'additional_special_tokens': sectoks})

Additional special tokens: ['[G]', '[C]', '[H]', '[B]', '[A]', '[E]', '[F]', '[D]']


8

In [46]:
sep = " " + tokenizer.sep_token + " "
df_train["inputs"] = df_train.sectok + sep + df_train.anchor + sep + df_train.target + sep + df_train.title + sep + df_train.anchor_context_targets
df_train["inputs"] = df_train["inputs"].apply(lambda x: x.lower())
df_train.head()

,id,anchor,target,context,score,section,anchor_map,context_map,anchor_context_map,score_map,kfold,code,title,anchor_context_targets,anc_ctx_tgt_len,sectok,inputs
18583,426b5d4ee52dfbba,reflection type liquid crystal display,reflective mode liquid display,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19157,d583a6c02fed7b2a,reflection type liquid crystal display,reflection mode lcd crystal,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19227,668eb746e5b96f9e,reflection type liquid crystal display,mobile,G02,0.25,G,555,89,555_89,1,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19221,b0e707f934a27619,reflection type liquid crystal display,reflection type crystal display,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19217,722fbf83a2054afa,reflection type liquid crystal display,liquid crystal device,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...


In [47]:
def tokenize_text(tokenizer, with_labels, row):
    encoding = tokenizer(
        text = row["inputs"],
        padding = False,
        truncation = True,
        # maximum possible sequence length (for inputs column). Sequences exceeding this length will be truncated
        max_length = 512
    )
    if with_labels:
        encoding["labels"] = row[config.train_run.label_col]
    return encoding

In [48]:
from functools import partial

preprocess_train_data = partial(tokenize_text, tokenizer, True)  
preprocess_test_data = partial(tokenize_text, tokenizer, False)  

In [49]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [50]:
def get_oof_preds(trainer, ds_val, df_val_fold):
    oof_outputs = trainer.predict(ds_val)
    oof_predictions = oof_outputs.predictions.reshape(-1)
    df_val_fold["val_preds"] = oof_predictions
    return df_val_fold

In [51]:
def clean_up():
    gc.collect()
    torch.cuda.empty_cache()
    utils.delete_checkpoints(config.training_args["output_dir"])
    print(f"deleted checkpoints as best model for {fold_str} saved already")
    # Empty the trash to clear gdrive disk space
    colab_utils.empty_gdrive_trash()

In [52]:
import gc
import custom_transformer_heads
import time

df_val_preds = pd.DataFrame()
tok_vocab = tokenizer.get_vocab()
exp_start_time = time.time()
for fold in range(config.train_run.num_folds):
    fold_str = f"fold{fold}"
    print(f"Running training for {fold_str}")
    if config.wandb.enabled:
        initialize_wandb(fold)
    df_train_fold, df_val_fold, ds_train, ds_val = utils.get_fold_ds(fold, df_train, preprocess_train_data)
    config.training_args["output_dir"] = config.paths.out_dir + fold_str
    training_args = TrainingArguments(**config.training_args)
    model = custom_transformer_heads.DebertaV2ForSeqClfMeanPooling.from_pretrained(
        config.train_run.transformer_checkpoint, 
        num_labels=config.train_run.num_labels
    )
    print(f"len(tokenizer_vocab) = {len(tok_vocab)}")
    model.resize_token_embeddings(len(tok_vocab))    
    trainer = Trainer(
        model=model,                         # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=ds_train,              # training dataset
        eval_dataset=ds_val,                 # evaluation dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
        data_collator=data_collator,
        tokenizer=tokenizer
    )
    trainer.train()
    trainer.save_model(config.training_args["output_dir"])
    df_val_fold = get_oof_preds(trainer, ds_val, df_val_fold) 
    # display(df_val_fold.head())
    df_val_preds = pd.concat([df_val_preds, df_val_fold], axis=0)
    # export the oof predictions to csv for later use in stacking    
    df_val_fold.to_csv(config.paths.val_preds_path + f"df_train_oof_preds_{MODEL_NAME}_{fold_str}.csv")
    print(f"Saved OOF predictions for fold {fold}")    
    del model, trainer
    clean_up()
    if not config.train_run.run_all_folds:
        break

exp_end_time = time.time()        

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running training for fold0


wandb: Currently logged in as: bkanupam. Use `wandb login --relogin` to force relogin


Parameter 'function'=<function Dataset.map.<locals>.decorate.<locals>.decorated at 0x7f4da7d31b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSeqClfMeanPooling: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2ForSeqClfMeanPooling from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSeqClfMeanPooling from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequen

len(tokenizer_vocab) = 128009


Epoch,Training Loss,Validation Loss,Pearson
0,No log,0.048141,0.737365
1,No log,0.027970,0.779183


Saved OOF predictions for fold 0
deleted checkpoints as best model for fold0 saved already
The below files were cleared from trash
[]


In [53]:
# Calculate the CV score
def calculate_cv_score(df_oof):
    predictions = df_oof['val_preds'].values
    labels = df_oof['score'].values
    eval_preds = predictions, labels
    cv_metric_dict = compute_metrics(eval_preds)
    return cv_metric_dict["pearson"]

In [58]:
import json

# save config file, log file, oof file etc. related to training run
exp_run_time = utils.asHours(exp_end_time - exp_start_time)
if config.train_run.run_all_folds:
        df_val_preds.to_csv(config.paths.val_preds_path + f"df_train_oof_preds_{MODEL_NAME}.csv")

cv_score = calculate_cv_score(df_val_preds)
print(f"cv_score = {cv_score}")
run_summary_dict = {
    "experiment": config.train_run.experiment_name,
    "cv": cv_score,
    "experiment_time": exp_run_time,
    "wandb_run": wandb.run.get_url() if config.wandb.enabled else None,
}
print(run_summary_dict)
if config.train_run.save_artifacts:
    run_summary_file = config.paths.out_dir + "/run_summary.json"
    with open(run_summary_file, "w") as f:
        json.dump(run_summary_dict, f, indent=4)

    # save run config yaml file (hydra config)
    exp_config_file = config.paths.out_dir + "/exp_config.yaml"
    with open(exp_config_file, "w") as fp:
        OmegaConf.save(config, fp)

    # upload artifacts and log final metrics to wandb
    if config.wandb.enabled:
        # Log config, cv score, run time to wandb
        wandb.log({"cv_score": cv_score, "exp_run_time": exp_run_time})    
        wandb.save(exp_config_file)        
        wandb.save(run_summary_file)


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


cv_score = 0.7791831694320728
{'experiment': '00_deberta-v3-small_baseline', 'cv': 0.7791831694320728, 'experiment_time': '0h:10m:50s', 'wandb_run': 'https://wandb.ai/bkanupam/USPPPM/runs/1rjhww9j'}


OSError: ignored